# Run different Ensemble methods, Oversampling and Feature selection and reduction.

In [1]:
# Data preparation

import pandas as pd
from sklearn import preprocessing
from IPython.display import display, HTML

df_loan200K=pd.read_csv('Data_Loans_200K.csv', header=0)
df_loan200K = df_loan200K.drop('Unnamed: 0', axis=1)

print(df_loan200K.shape)
cols_loan200K = df_loan200K.columns
 
    
print('Column Name, DataTypes, MissingValues in Loan 200K CSV\n')
for i in cols_loan200K:
    print(i,',', df_loan200K[i].dtype , ',', df_loan200K[i].isnull().any())

##If Yes, fill in missing values by mean values or most frequent nominal values.

df_loan200K["Credit Score"].fillna(df_loan200K["Credit Score"].mean(), inplace=True)
df_loan200K["Annual Income"].fillna(df_loan200K["Annual Income"].mean(), inplace=True)
df_loan200K["Years in current job"].fillna(df_loan200K["Years in current job"].mode().iloc[0], inplace=True)
df_loan200K["Months since last delinquent"].fillna(df_loan200K["Months since last delinquent"].mean(), inplace=True)
df_loan200K["Maximum Open Credit"].fillna(df_loan200K["Maximum Open Credit"].mean(), inplace=True)
df_loan200K["Bankruptcies"].fillna(df_loan200K["Bankruptcies"].mean(), inplace=True)
df_loan200K["Tax Liens"].fillna(df_loan200K["Tax Liens"].mean(), inplace=True)

print('\nColumn Name, DataTypes, MissingValues after filling with Mean and Frequent repeated nominal value in Loan 200K CSV\n')
for i in cols_loan200K:
    print(i,',', df_loan200K[i].dtype , ',', df_loan200K[i].isnull().any()) 
   
# encode labels
y = df_loan200K['Term'] # define label as nominal values
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y) # encode nominal labels to integers #####################################
df_loan200K['Term'] = y_encoded

# Data preprocessing ################################################################################
print('Column Datatypes:\n',df_loan200K.dtypes)

# convert all nominal variables to binary variables
df_num=df_loan200K.copy(deep=True) 
# create new binary columns
df_dummies=pd.get_dummies(df_num[['Loan Status','Years in current job','Home Ownership','Purpose']])
# add them to dataframe
df_num=df_num.join(df_dummies)
# drop original columns
df_num=df_num.drop('Loan Status',axis=1)
df_num=df_num.drop('Years in current job',axis=1)
df_num=df_num.drop('Home Ownership', axis=1)
df_num=df_num.drop('Purpose', axis=1)

# drop extra binary columns, since we only need N-1 binary columns
df_num=df_num.drop('Loan Status_Charged Off', axis=1)
df_num=df_num.drop('Years in current job_10+ years', axis=1)
df_num=df_num.drop('Home Ownership_Own Home', axis=1)
df_num=df_num.drop('Purpose_wedding', axis=1)

# print out and display dataframe as table in HTML
display(HTML(df_num.head(10).to_html()))

(200000, 17)
Column Name, DataTypes, MissingValues in Loan 200K CSV

Loan Status , object , False
Current Loan Amount , int64 , False
Term , object , False
Credit Score , float64 , True
Annual Income , float64 , True
Years in current job , object , True
Home Ownership , object , False
Purpose , object , False
Monthly Debt , float64 , False
Years of Credit History , float64 , False
Months since last delinquent , float64 , True
Number of Open Accounts , int64 , False
Number of Credit Problems , int64 , False
Current Credit Balance , int64 , False
Maximum Open Credit , float64 , True
Bankruptcies , float64 , True
Tax Liens , float64 , True

Column Name, DataTypes, MissingValues after filling with Mean and Frequent repeated nominal value in Loan 200K CSV

Loan Status , object , False
Current Loan Amount , int64 , False
Term , object , False
Credit Score , float64 , False
Annual Income , float64 , False
Years in current job , object , False
Home Ownership , object , False
Purpose , object ,

In [2]:
# 02. Bagging meta-estimator

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score

y = df_num['Term']
x = df_num.drop('Term', axis=1)

# by hold-out evaluation
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
tree = DecisionTreeClassifier(criterion='gini', max_depth=10, ccp_alpha = 0.1)
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8, random_state=1)
clf=bag.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
pre=precision_score(y_test, y_pred, average='macro')
rec=recall_score(y_test, y_pred, average='macro')
print("Bagging meta-estimator accuracy by 10-fold Cross Validation: acc = ",acc, "precision = ", pre, "recall =", rec)

# by hold-out evaluation
tree = DecisionTreeClassifier(criterion='gini', max_depth=10, ccp_alpha = 0.1)
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8, max_features=0.8)
clf=bag.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
pre=precision_score(y_test, y_pred, average='macro')
rec=recall_score(y_test, y_pred, average='macro')
print("Bagging meta-estimator accuracy by 10-fold Cross Validation: acc = ",acc, "precision = ", pre, "recall =", rec)

C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Bagging meta-estimator accuracy by 10-fold Cross Validation: acc =  0.61126 precision =  0.30563 recall = 0.5
Bagging meta-estimator accuracy by 10-fold Cross Validation: acc =  0.61126 precision =  0.30563 recall = 0.5


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [3]:
# 03. Random Forset 
from sklearn.ensemble import RandomForestClassifier

# by hold-out evaluation Bagging meta-estimator
randForest = RandomForestClassifier(criterion='gini', max_depth=10, ccp_alpha = 0.1, n_estimators=100, max_samples=0.8, random_state=1)
clf=randForest.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
pre=precision_score(y_test, y_pred, average='macro')
rec=recall_score(y_test, y_pred, average='macro')
print("Forests of randomized trees accuracy by 10-fold Cross Validation: acc = ",acc, "precision = ", pre, "recall =", rec)

# by hold-out evaluation Bagging meta-estimator
randForest = RandomForestClassifier(criterion='gini', max_depth=10, ccp_alpha = 0.1, n_estimators=100, max_samples=0.8, max_features=0.8)
clf=randForest.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
pre=precision_score(y_test, y_pred, average='macro')
rec=recall_score(y_test, y_pred, average='macro')
print("Forests of randomized trees accuracy by 10-fold Cross Validation: acc = ",acc, "precision = ", pre, "recall =", rec)

C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Forests of randomized trees accuracy by 10-fold Cross Validation: acc =  0.61126 precision =  0.30563 recall = 0.5
Forests of randomized trees accuracy by 10-fold Cross Validation: acc =  0.61126 precision =  0.30563 recall = 0.5


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
# 04. AdaBoosting

from sklearn.ensemble import AdaBoostClassifier

# by hold-out evaluation AdaBoost Classifier
tree = DecisionTreeClassifier(criterion='gini', max_depth=10, ccp_alpha = 0.1)
adaBoost = AdaBoostClassifier(tree, n_estimators=100, learning_rate=1.0, algorithm='SAMME', random_state=1)
clf=adaBoost.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
pre=precision_score(y_test, y_pred, average='macro')
rec=recall_score(y_test, y_pred, average='macro')
print("AdaBoosting accuracy by 10-fold Cross Validation: acc = ",acc, "precision = ", pre, "recall =", rec)

# by hold-out evaluation  AdaBoost Classifier
tree = DecisionTreeClassifier(criterion='gini', max_depth=10, ccp_alpha = 0.1)
adaBoost = AdaBoostClassifier(tree, n_estimators=100, learning_rate=1.0, algorithm='SAMME.R', random_state=1 )
clf=adaBoost.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
pre=precision_score(y_test, y_pred, average='macro')
rec=recall_score(y_test, y_pred, average='macro')
print("AdaBoosting accuracy by 10-fold Cross Validation: acc = ",acc, "precision = ", pre, "recall =", rec)

C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AdaBoosting accuracy by 10-fold Cross Validation: acc =  0.61126 precision =  0.30563 recall = 0.5
AdaBoosting accuracy by 10-fold Cross Validation: acc =  0.61126 precision =  0.30563 recall = 0.5


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
# 05. Gradient Boosting 
from sklearn.ensemble import GradientBoostingClassifier

# by hold-out evaluation
clf = GradientBoostingClassifier(n_estimators=100, random_state=1, learning_rate=1.0, criterion='squared_error') 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
pre=precision_score(y_test, y_pred, average='macro')
rec=recall_score(y_test, y_pred, average='macro')
print("GradientBoosting Accuracy by N-fold Cross Validation: acc = ",acc, "precision = ", pre, "recall =", rec)

GradientBoosting Accuracy by N-fold Cross Validation: acc =  0.66224 precision =  0.6418726730410425 recall = 0.6366097928385117


In [6]:
# 06. Histogram Gradient Boosting
from sklearn.ensemble import HistGradientBoostingClassifier

# by hold-out evaluation
clf = HistGradientBoostingClassifier(random_state=1, learning_rate=1.0, max_depth=10) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
pre=precision_score(y_test, y_pred, average='macro')
rec=recall_score(y_test, y_pred, average='macro')
print("HistGradientBoosting Accuracy by N-fold Cross Validation: acc = ",acc, "precision = ", pre, "recall =", rec)

HistGradientBoosting Accuracy by N-fold Cross Validation: acc =  0.6576 precision =  0.639879036637064 recall = 0.640025006454348


In [7]:
# 07. Oversampling

from collections import Counter
from imblearn.over_sampling import RandomOverSampler


ros = RandomOverSampler(random_state=10)
ros.fit(x, y)
print('\nOriginal dataset shape {}'.format(Counter(y)))
# by hold-out evaluation
clf=DecisionTreeClassifier(criterion='gini', max_depth=10, ccp_alpha = 0.1)
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
pre=precision_score(y_test, y_pred, average='macro')
rec=recall_score(y_test, y_pred, average='macro')
print('By 10-fold Cross Validation: Original Decision tree accuracy = ',acc, ', precison = ', pre, ', recall = ', rec)

x_resampled, y_resampled = ros.fit_resample(x, y)
print('After oversampling dataset shape {}'.format(Counter(y_resampled)))

# by hold-out evaluation
x_rs_train, x_rs_test, y_rs_train, y_rs_test = train_test_split(x_resampled, y_resampled, test_size=0.25)
clf=DecisionTreeClassifier(criterion='gini', max_depth=10, ccp_alpha = 0.1)
clf=clf.fit(x_rs_train, y_rs_train)
y_rs_pred=clf.predict(x_rs_test)
acc=accuracy_score(y_rs_test, y_rs_pred)
pre=precision_score(y_rs_test, y_rs_pred, average='macro')
rec=recall_score(y_rs_test, y_rs_pred, average='macro')
print('By 10-fold Cross Validation: Oversampling Decision tree accuracy = ',acc, ', precison = ', pre, ', recall = ', rec)


Original dataset shape Counter({1: 122238, 0: 77762})


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


By 10-fold Cross Validation: Original Decision tree accuracy =  0.61126 , precison =  0.30563 , recall =  0.5
After oversampling dataset shape Counter({1: 122238, 0: 122238})
By 10-fold Cross Validation: Oversampling Decision tree accuracy =  0.49788118261097203 , precison =  0.24894059130548601 , recall =  0.5


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
# 08. Feature selection and reduction

from scipy.stats import f_oneway

X = df_num.drop('Term', axis=1)
print('\nSelected features by ANOVA:')
for col in X.columns:
    group1 = df_num[df_num['Term']==0][col]
    group2 = df_num[df_num['Term']==1][col]
    group3 = df_num[df_num['Term']==2][col]
    group4 = df_num[df_num['Term']==3][col]
    stat, p = f_oneway(group1, group2, group3, group4)
    if p < 0.01:
        print(col)
        
# ANOVA method

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

selector = SelectKBest(f_classif, k=10).fit(x, y)
print('\nSelected features by ANOVA:', selector.get_feature_names_out())

X_reduced = x[selector.get_feature_names_out()]

# print out and display dataframe as table in HTML
display(HTML(X_reduced.head(10).to_html()))


selector = SelectKBest(f_classif, k=10).fit(x_resampled, y_resampled)
print('\nSelected features by ANOVA:', selector.get_feature_names_out())

X_resampled_reduced = x_resampled[selector.get_feature_names_out()]

# print out and display dataframe as table in HTML
display(HTML(X_resampled_reduced.head(10).to_html()))


C:\Users\vikas\anaconda3\lib\site-packages\scipy\stats\stats.py:3621: F_onewayBadInputSizesWarning: at least one input has length 0
  warnings.warn(F_onewayBadInputSizesWarning('at least one input '



Selected features by ANOVA:

Selected features by ANOVA: ['Current Loan Amount' 'Credit Score' 'Years of Credit History'
 'Number of Open Accounts' 'Number of Credit Problems' 'Bankruptcies'
 'Tax Liens' 'Loan Status_Fully Paid' 'Home Ownership_Rent'
 'Purpose_Debt Consolidation']



Selected features by ANOVA: ['Current Loan Amount' 'Credit Score' 'Years of Credit History'
 'Number of Open Accounts' 'Number of Credit Problems' 'Bankruptcies'
 'Tax Liens' 'Loan Status_Fully Paid' 'Home Ownership_Rent'
 'Purpose_Debt Consolidation']


In [9]:
# 09. Decision tree classifier after feature reduction.

# by hold-out evaluation
x_train, x_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.25)
clf=DecisionTreeClassifier(criterion='gini', max_depth=10, ccp_alpha = 0.1)
precision = make_scorer(precision_score, average='macro')
recall = make_scorer(recall_score, average='macro')
acc=cross_val_score(clf, X_reduced, y, cv=10, scoring='accuracy').mean()
pre=cross_val_score(clf, X_reduced, y, cv=10, scoring=precision).mean()
rec=cross_val_score(clf, X_reduced, y, cv=10, scoring=recall).mean()
print('By 10-fold Cross Validation: Original Decision tree accuracy = ',acc, ', precison = ', pre, ', recall = ', rec)

# by hold-out evaluation
x_train, x_test, y_train, y_test = train_test_split(X_resampled_reduced, y_resampled, test_size=0.25)
clf=DecisionTreeClassifier(criterion='gini', max_depth=10, ccp_alpha = 0.1)
clf=clf.fit(x_rs_train, y_rs_train)
y_rs_pred=clf.predict(x_rs_test)
acc=accuracy_score(y_rs_test, y_rs_pred)
pre=precision_score(y_rs_test, y_rs_pred, average='macro')
rec=recall_score(y_rs_test, y_rs_pred, average='macro')
print('By 10-fold Cross Validation: Oversampling Decision tree accuracy = ',acc, ', precison = ', pre, ', recall = ', rec)

C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedM

By 10-fold Cross Validation: Original Decision tree accuracy =  0.61119 , precison =  0.305595 , recall =  0.5
By 10-fold Cross Validation: Oversampling Decision tree accuracy =  0.49788118261097203 , precison =  0.24894059130548601 , recall =  0.5


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
